
TODO: 
Check out the smart driver
- https://dev.to/franckpachot/cluster-aware-psycopg2-405i
- https://github.com/yugabyte/psycopg2

Isolation levels
- https://dev.to/yugabyte/read-committed-is-a-must-for-postgres-compatible-distributed-sql-databases-59pf

Fast path (WAL/ IntentsDB, RegularDB)
- https://dev.to/yugabyte/yugabytedb-fast-path-write-51hm

https://dev.to/franckpachot

<div style="width:100%; background-color: #000041"><a target="_blank" href="http://university.yugabyte.com"><img src="assets/YBU_Logo.png" /></a></div>

> **YugabyteDB YSQL Development**
>
> Enroll for free  [Yugabyte University](https://university.yugabyte.com/).
>

# Requirements to run this locally
Here are the requirements to run this notebook locally:

- Download and install YugabyteDB
- Update the `env_vars.env` file
- Install Python 3.8 or higher
  
## Download and install YugabyteDB
To run this notebook locally in VS Code on MacOs or Linux, you must download and install YugabyteDB:
### MacOS 
You can download YugabyteDB for the MacOS at [https://download.yugabyte.com/#macos](https://download.yugabyte.com/#macos).

### Linux
You can download YugabyteDB for Linux at [https://download.yugabyte.com/#linux](https://download.yugabyte.com/#linux).

### Docker
You can download YugabyteDB for Docker at [https://download.yugabyte.com/#docker](https://download.yugabyte.com/#docker).

## Update the `env_vars.env` file
After installing YugabyteDB locally, you need to update these paths in `env_vars.env`:
```
MY_YB_PATH_LOCAL_2.16=/Users/seth/yugabyte/yugabyte-2.16.3.0
MY_YB_PATH_LOCAL=/Users/seth/yugabyte/yugabyte-2.17.3.0
MY_YB_PATH_DATA_LOCAL=/Users/seth/yugabyte-data
```

## Install Python 3.8 or higher
For MacOs, PyEnv is probably the easiest way to install. You can learn more at:
- [https://github.com/pyenv/pyenv](https://github.com/pyenv/pyenv)
- [https://realpython.com/intro-to-pyenv/](https://realpython.com/intro-to-pyenv/)

---
# Update the notebook dependencies.

%%markdown
<style>
    .rendered_html table {
        border-collapse: collapse;
        margin: auto;
    }
    .rendered_html th, .rendered_html td {
        border: 1px solid black;
        padding: 8px;
        text-align: left;
    }
    .rendered_html th {
        background-color: #f2f2f2;
    }
</style>


In [17]:
%%html
<style>
    .rendered_html table {
        border-collapse: collapse;
        margin: auto;
    }
    .rendered_html th, .rendered_html td {
        border: 1px solid black;
        padding: 8px;
        text-align: left;
    }
    .rendered_html th {
        background-color: #f2f2f2;
    }
</style>
<table id="rendered_html" >
    <tr>
      <th>Month</th>
      <th>Savings</th>
    </tr>
    <tr>
      <td>January</td>
      <td>$100</td>
    </tr>
    <tr>
      <td>February</td>
      <td>$80</td>
    </tr>
  </table>

Month,Savings
January,$100
February,$80


In [ ]:
from IPython.core.display import HTML

# Define your custom CSS
style = """
<style>
    .rendered_html table {
        border-collapse: collapse;
        margin: auto;
    }
    .rendered_html th, .rendered_html td {
        border: 1px solid black;
        padding: 8px;
        text-align: left;
    }
    .rendered_html th {
        background-color: #f2f2f2;
    }
</style>
"""

# Inject the CSS into the notebook
display(HTML(style))


In [ ]:
!pip install --upgrade pip
!pip install ipython-sql~=0.5 --upgrade
!pip install psycopg2-binary
!pip install SQLAlchemy --upgrade 
!pip install sqlparse
!pip install html2text
!pip install jq

### Create the notebook variables 

> IMPORTANT!
> 
> Do NOT skip editing and running this cell. 
> 

The following Python cell creates and stores the Jupyter notebook variables.
- First, edit the script to select how to assign a value to `MY_SUDO`.
- To run the script, select Execute Cell (Play Arrow in the cell side bar).
- Verify the accuracy of the output values

> About sudo and MacOS:
> 
> You'll need to supply your sudo password to create the loopback addresses.
>
> 

In [ ]:
# Env variables for Notebook
import os

env_vars = !cat env_vars.env
for var in env_vars:
    key, value = var.split('=')
    os.environ[key] = value
 
### BEGIN: Edit your MY_SUDO

# Leave it blank forcing a strategy for notebook execution
#MY_SUDO=''

# Specify your sudo password in the cell
#MY_SUDO='psswrd'

# env_vars defines ENV_SUDO='psswrd'
#MY_SUDO=os.environ.get('MY_SUDO')

# bash_profile exports the variable, e.g., export GLOBAL_SUDO='psswrd' 
MY_SUDO=os.environ.get('GLOBAL_SUDO')

### END: MY_SUDO


# Create and assign notebook variables
# env_vars defines the following
MY_DB_NAME=os.environ.get('MY_DB_NAME')

MY_YB_PATH=os.environ.get('MY_YB_PATH_LOCAL')
MY_YB_PATH_DATA=os.environ.get('MY_YB_PATH_DATA_LOCAL')

MY_HOST_IPv4_01=os.environ.get('MY_HOST_IPv4_01')
MY_HOST_IPv4_02=os.environ.get('MY_HOST_IPv4_02')
MY_HOST_IPv4_03=os.environ.get('MY_HOST_IPv4_03')
MY_TSERVER_WEBSERVER_PORT=os.environ.get('MY_TSERVER_WEBSERVER_PORT')

MY_NOTEBOOK_DIR=os.getcwd()
MY_NOTEBOOK_DATA_FOLDER=MY_NOTEBOOK_DIR +'/data'
MY_NOTEBOOK_UTILS_FOLDER=MY_NOTEBOOK_DIR + '/utils'
MY_UTIL_FUNCTIONS_FILE=os.environ.get("MY_UTIL_FUNCTIONS_FILE")
# Store the note book values for other notebooks to use
%store MY_SUDO
%store MY_DB_NAME
%store MY_YB_PATH
%store MY_YB_PATH_DATA
%store MY_HOST_IPv4_01
%store MY_HOST_IPv4_02
%store MY_HOST_IPv4_03
%store MY_TSERVER_WEBSERVER_PORT

### Create the loopback host aliases for MacOS
> Important!
> 
> The script requires the password for `sudo`.
> 
> The bash cell utilizes the notebook variable, `MY_SUDO`.
> 
> If you did not define the `MY_SUDO` notebook variable using one of the suggested options, you can copy this script into the terminal, which is interactive, and manually supply the password.

The YugabyteDB local cluster for this lab requires that your create loop back aliases for your local host (127.0.0.1). The bash cell does the following:
- Deletes existing aliases
- Creates the loopback aliases from `127.0.0.2` to `127.0.0.7`

In [ ]:
%%bash -s "$MY_SUDO"  # ifconfig aliases
MY_SUDO=${1}

if ifconfig lo0 | grep 127.0.0.[2-7] > /dev/null
then
    echo ${MY_SUDO} | sudo -S ifconfig lo0 delete 127.0.0.2
    echo ${MY_SUDO} | sudo -S ifconfig lo0 delete 127.0.0.3
    echo ${MY_SUDO} | sudo -S ifconfig lo0 delete 127.0.0.4
    echo ${MY_SUDO} | sudo -S ifconfig lo0 delete 127.0.0.5
    echo ${MY_SUDO} | sudo -S ifconfig lo0 delete 127.0.0.6
    echo ${MY_SUDO} | sudo -S ifconfig lo0 delete 127.0.0.7
fi

echo ${MY_SUDO} | sudo -S ifconfig lo0 alias 127.0.0.2
echo ${MY_SUDO} | sudo -S ifconfig lo0 alias 127.0.0.3
echo ${MY_SUDO} | sudo -S ifconfig lo0 alias 127.0.0.4
echo ${MY_SUDO} | sudo -S ifconfig lo0 alias 127.0.0.5
echo ${MY_SUDO} | sudo -S ifconfig lo0 alias 127.0.0.6
echo ${MY_SUDO} | sudo -S ifconfig lo0 alias 127.0.0.7

echo ${MY_SUDO} | sudo ifconfig lo0

### Create a YugabyteDB local cluster
- If a YugabyteDB local cluster is running, it will be stopped and destroyed using `yb-ctl`
- To avoid the port 9000 conflict, specifies port `TSERVER_WEBSERVER_PORT` as `8200`

In [ ]:
%%bash
process_ids=$(pgrep -f 'yuga')

# Check if any processes were found
if [ -z "$process_ids" ]; then
    echo "No processes found containing 'yuga' in the file path."
    exit 0
fi

# Kill the processes
for pid in $process_ids; do
    kill $pid
    echo "Killed process $pid"
done

In [ ]:
%%bash
### Grep port 9000 for conflict
 lsof -nP -iTCP -sTCP:LISTEN | grep 8890
## this should be juypter

In [ ]:
%%bash -s "$MY_YB_PATH"  "$MY_YB_PATH_DATA"  "$MY_TSERVER_WEBSERVER_PORT"  # yb-ctl create
YB_PATH=${1}
YB_PATH_DATA=${2}
TSERVER_WEBSERVER_PORT=${3}

cd $YB_PATH


## this should be juypter


./bin/yugabyted start  \
        --advertise_address="127.0.0.1"  \
        --base_dir=${YB_PATH_DATA}/node-1  \
        --cloud_location=cloud1.region1.zone1  \
        --fault_tolerance=zone  \
        --master_flags="yb_num_shards_per_tserver=1,ysql_num_shards_per_tserver=1,ysql_beta_features=true,ysql_enable_packed_row=false"  \
        --master_webserver_port=7000  \
        --tserver_flags="yb_num_shards_per_tserver=1,ysql_num_shards_per_tserver=1,ysql_beta_features=true,ysql_enable_packed_row=false"  \
        --tserver_webserver_port=8200 > dev\null
        
# ./bin/yugabyted start  \
#         --advertise_address="127.0.0.2"  \
#         --base_dir=${YB_PATH_DATA}/node-2 \
#         --cloud_location=cloud1.region1.zone2  \
#         --fault_tolerance=zone  \
#         --join="127.0.0.1"  \
#         --master_flags="yb_num_shards_per_tserver=1,ysql_num_shards_per_tserver=1,ysql_beta_features=true,ysql_enable_packed_row=false"  \
#         --master_webserver_port=7000  \
#         --tserver_flags="yb_num_shards_per_tserver=1,ysql_num_shards_per_tserver=1,ysql_beta_features=true,ysql_enable_packed_row=false"  \
#         --tserver_webserver_port=8200

#    ./bin/yugabyted start  \
#         --advertise_address="127.0.0.3"  \
#         --base_dir=${YB_PATH_DATA}/node-3  \
#         --cloud_location=cloud1.region1.zone3  \
#         --fault_tolerance=zone  \
#         --join="127.0.0.1"  \
#         --master_flags="yb_num_shards_per_tserver=1,ysql_num_shards_per_tserver=1,ysql_beta_features=true,ysql_enable_packed_row=false"  \
#         --master_webserver_port=7000  \
#         --tserver_flags="yb_num_shards_per_tserver=1,ysql_num_shards_per_tserver=1,ysql_beta_features=true,ysql_enable_packed_row=false"  \
#         --tserver_webserver_port=8200

# ./bin/yugabyted configure data_placement --fault_tolerance=zone

# ./bin/yugabyted status

In the above, you'll see the status of the YugabyteDB cluster running locally.

### YCQL API

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_TSERVER_WEBSERVER_PORT"  # yb-ctl create
YB_PATH=${1}
TSERVER_WEBSERVER_PORT=${2}

cd $YB_PATH

# select version
./bin/ycqlsh -e "
  select cql_version from system.local;
  "  

### YSQL API

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME"  # create database
YB_PATH=${1}
DB_NAME=${2}

cd $YB_PATH

# select version
./bin/ysqlsh -d yugabyte -c "
   select version();
"  

create database

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME"  # create database
YB_PATH=${1}
DB_NAME=${2}

cd $YB_PATH

# terminate connections, drop, and create
./bin/ysqlsh -d yugabyte -c "select pg_terminate_backend(pid) from pg_stat_activity where datname = '"${DB_NAME}"';"  
./bin/ysqlsh -d yugabyte -c "drop database if exists "${DB_NAME}";"  
./bin/ysqlsh -d yugabyte -c "create database "${DB_NAME}";" 

# list dbs
./bin/ysqlsh -d yugabyte -c "\l"

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME" "$MY_NOTEBOOK_UTILS_FOLDER" "$MY_UTIL_FUNCTIONS_FILE"  # util functions
YB_PATH=${1}
DB_NAME=${2}
UTILS_FOLDER=${3}
UTIL_FUNCTIONS_FILE=${4}


#ls $UTIL_FOLDER
UTIL_FUNCTIONS_FILE_PATH=${UTILS_FOLDER}/${UTIL_FUNCTIONS_FILE}

cd $YB_PATH

# Functions file
./bin/ysqlsh -q -d ${DB_NAME} -f ${UTIL_FUNCTIONS_FILE_PATH} 
sleep 1;

# Describe functions
./bin/ysqlsh -d ${DB_NAME} -c "\df public.fn_*"

### SQL Magic

In [ ]:
# Connect to db_ybu
# Inspiration from https://medium.com/analytics-vidhya/postgresql-integration-with-jupyter-notebook-deb97579a38d
import psycopg2
import sqlalchemy as alc
from sqlalchemy import create_engine

# env_var.env
db_host=MY_HOST_IPv4_01
db_name=MY_DB_NAME

connection_str='postgresql+psycopg2://yugabyte@'+db_host+':5433/'+db_name
print(connection_str)
engine = create_engine(connection_str)

In [ ]:
%reload_ext sql 
# creates connection for sql magic
%sql {connection_str}

---


## Fast Path

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME" "$MY_NOTEBOOK_UTILS_FOLDER" "$MY_UTIL_FUNCTIONS_FILE"  # util functions
MY_TSERVER_LOGS=/Users/seth/yugabyte-data/node-1/yb-data/tserver/logs  
ls $MY_TSERVER_LOGS

tail -f $MY_TSERVER_LOGS/yb-tserver.INFO | grep -E '^ | tablet.cc:'


---

show hash


In [ ]:
%%sql 
select pg_sleep(5), replace(current_setting('listen_addresses'),'0.0.0.0', host(inet_server_addr())::text);

In [ ]:
%%sql 
drop table if exists demo;


In [ ]:
%%sql 

create table if not exists demo(id bigint primary key, value text);


In [ ]:
%%sql 

create unique index idx_unq_demo on demo(value);

In [ ]:
%%sql 
insert into demo values (300,'hey now3');

In [7]:
%%sql
explain analyze select id from demo where value = 'hey now3';

 * postgresql+psycopg2://yugabyte@127.0.0.1:5433/db_ybu
5 rows affected.


QUERY PLAN
Index Scan using idx_unq_demo on demo (cost=0.00..4.12 rows=1 width=8) (actual time=0.881..0.883 rows=1 loops=1)
Index Cond: (value = 'hey now3'::text)
Planning Time: 3.660 ms
Execution Time: 0.914 ms
Peak Memory Usage: 8 kB


In [ ]:
%%sql 
update demo set value='question' where id 1=1;;

```
I0731 12:49:46.371345 129552384 tablet.cc:1495] T bb4179e058cf4fcc84da1573eeed5be5 P d3f71cf04f7844ba81a9753135831117: Wrote 1 key/value pairs to kRegular RocksDB:
  T bb4179e058cf4fcc84da1573eeed5be5 P d3f71cf04f7844ba81a9753135831117 [R]: 1. PutCF: SubDocKey(DocKey(0xbf4f, [42], []), [ColumnId(1); HT{ physical: 1690832986370808 }]) => "question"
```

In [ ]:
%%sql 
delete from demo where 1=1;

```
I0731 12:51:34.516342 129552384 tablet.cc:1495] T bb4179e058cf4fcc84da1573eeed5be5 P d3f71cf04f7844ba81a9753135831117: Wrote 1 key/value pairs to kRegular RocksDB:
  T bb4179e058cf4fcc84da1573eeed5be5 P d3f71cf04f7844ba81a9753135831117 [R]: 1. PutCF: SubDocKey(DocKey(0xbf4f, [42], []), [HT{ physical: 1690833094515929 }]) => DEL
```


Now, a distributed transaction.

In [ ]:
%%sql 
insert into demo values (1,'HitchHiker'),(2,'Restaurant');

```
I0731 13:32:18.626562 22200320 tablet.cc:1495] T ab874dea16d64a14892c2f3fea29a981 P ade1a2d814884e498b1f69a1d1be1041: Wrote 15 key/value pairs to kIntents RocksDB:
  T ab874dea16d64a14892c2f3fea29a981 P ade1a2d814884e498b1f69a1d1be1041 [I]: 1. PutCF: TXN META 45f1546f-1634-45a1-aa7a-80a80a0b84a4 => { transaction_id: 45f1546f-1634-45a1-aa7a-80a80a0b84a4 isolation: SNAPSHOT_ISOLATION status_tablet: 6f7bcec790f64814b036d261d65f9a3a priority: 13210515334273129354 start_time: { physical: 1690835538625177 } locality: GLOBAL old_status_tablet:  external_transaction: 0}
  T ab874dea16d64a14892c2f3fea29a981 P ade1a2d814884e498b1f69a1d1be1041 [I]: 2. PutCF: SubDocKey(DocKey(0xeda9, [1], []), [SystemColumnId(0)]) [kStrongRead, kStrongWrite] HT{ physical: 1690835538626171 } => TransactionId(45f1546f-1634-45a1-aa7a-80a80a0b84a4) WriteId(0) null
  T ab874dea16d64a14892c2f3fea29a981 P ade1a2d814884e498b1f69a1d1be1041 [I]: 3. PutCF: TXN REV 45f1546f-1634-45a1-aa7a-80a80a0b84a4 HT{ physical: 1690835538626171 } => SubDocKey(DocKey(0xeda9, [1], []), [SystemColumnId(0)]) [kStrongRead, kStrongWrite] HT{ physical: 1690835538626171 }
  T ab874dea16d64a14892c2f3fea29a981 P ade1a2d814884e498b1f69a1d1be1041 [I]: 4. PutCF: SubDocKey(DocKey(0xeda9, [1], []), [ColumnId(1)]) [kStrongRead, kStrongWrite] HT{ physical: 1690835538626171 w: 1 } => TransactionId(45f1546f-1634-45a1-aa7a-80a80a0b84a4) WriteId(1) "HitchHiker"
  T ab874dea16d64a14892c2f3fea29a981 P ade1a2d814884e498b1f69a1d1be1041 [I]: 5. PutCF: TXN REV 45f1546f-1634-45a1-aa7a-80a80a0b84a4 HT{ physical: 1690835538626171 w: 1 } => SubDocKey(DocKey(0xeda9, [1], []), [ColumnId(1)]) [kStrongRead, kStrongWrite] HT{ physical: 1690835538626171 w: 1 }
  T ab874dea16d64a14892c2f3fea29a981 P ade1a2d814884e498b1f69a1d1be1041 [I]: 6. PutCF: SubDocKey(DocKey(0xcfaa, [2], []), [SystemColumnId(0)]) [kStrongRead, kStrongWrite] HT{ physical: 1690835538626171 w: 2 } => TransactionId(45f1546f-1634-45a1-aa7a-80a80a0b84a4) WriteId(2) null
  T ab874dea16d64a14892c2f3fea29a981 P ade1a2d814884e498b1f69a1d1be1041 [I]: 7. PutCF: TXN REV 45f1546f-1634-45a1-aa7a-80a80a0b84a4 HT{ physical: 1690835538626171 w: 2 } => SubDocKey(DocKey(0xcfaa, [2], []), [SystemColumnId(0)]) [kStrongRead, kStrongWrite] HT{ physical: 1690835538626171 w: 2 }
  T ab874dea16d64a14892c2f3fea29a981 P ade1a2d814884e498b1f69a1d1be1041 [I]: 8. PutCF: SubDocKey(DocKey(0xcfaa, [2], []), [ColumnId(1)]) [kStrongRead, kStrongWrite] HT{ physical: 1690835538626171 w: 3 } => TransactionId(45f1546f-1634-45a1-aa7a-80a80a0b84a4) WriteId(3) "Restaurant"
  T ab874dea16d64a14892c2f3fea29a981 P ade1a2d814884e498b1f69a1d1be1041 [I]: 9. PutCF: TXN REV 45f1546f-1634-45a1-aa7a-80a80a0b84a4 HT{ physical: 1690835538626171 w: 3 } => SubDocKey(DocKey(0xcfaa, [2], []), [ColumnId(1)]) [kStrongRead, kStrongWrite] HT{ physical: 1690835538626171 w: 3 }
  T ab874dea16d64a14892c2f3fea29a981 P ade1a2d814884e498b1f69a1d1be1041 [I]: 10. PutCF: SubDocKey(DocKey(0xcfaa, [2], []), []) [kWeakRead, kWeakWrite] HT{ physical: 1690835538626171 w: 4 } => TransactionId(45f1546f-1634-45a1-aa7a-80a80a0b84a4) none
  T ab874dea16d64a14892c2f3fea29a981 P ade1a2d814884e498b1f69a1d1be1041 [I]: 11. PutCF: TXN REV 45f1546f-1634-45a1-aa7a-80a80a0b84a4 HT{ physical: 1690835538626171 w: 4 } => SubDocKey(DocKey(0xcfaa, [2], []), []) [kWeakRead, kWeakWrite] HT{ physical: 1690835538626171 w: 4 }
  T ab874dea16d64a14892c2f3fea29a981 P ade1a2d814884e498b1f69a1d1be1041 [I]: 12. PutCF: SubDocKey(DocKey(0xeda9, [1], []), []) [kWeakRead, kWeakWrite] HT{ physical: 1690835538626171 w: 5 } => TransactionId(45f1546f-1634-45a1-aa7a-80a80a0b84a4) none
  T ab874dea16d64a14892c2f3fea29a981 P ade1a2d814884e498b1f69a1d1be1041 [I]: 13. PutCF: TXN REV 45f1546f-1634-45a1-aa7a-80a80a0b84a4 HT{ physical: 1690835538626171 w: 5 } => SubDocKey(DocKey(0xeda9, [1], []), []) [kWeakRead, kWeakWrite] HT{ physical: 1690835538626171 w: 5 }
  T ab874dea16d64a14892c2f3fea29a981 P ade1a2d814884e498b1f69a1d1be1041 [I]: 14. PutCF: SubDocKey(DocKey([], []), []) [kWeakRead, kWeakWrite] HT{ physical: 1690835538626171 w: 6 } => TransactionId(45f1546f-1634-45a1-aa7a-80a80a0b84a4) none
  T ab874dea16d64a14892c2f3fea29a981 P ade1a2d814884e498b1f69a1d1be1041 [I]: 15. PutCF: TXN REV 45f1546f-1634-45a1-aa7a-80a80a0b84a4 HT{ physical: 1690835538626171 w: 6 } => SubDocKey(DocKey([], []), []) [kWeakRead, kWeakWrite] HT{ physical: 1690835538626171 w: 6 }
I0731 13:32:18.627648 22200320 tablet.cc:1495] T ab874dea16d64a14892c2f3fea29a981 P ade1a2d814884e498b1f69a1d1be1041: Wrote 4 key/value pairs to kRegular RocksDB:
  T ab874dea16d64a14892c2f3fea29a981 P ade1a2d814884e498b1f69a1d1be1041 [R]: 1. PutCF: SubDocKey(DocKey(0xeda9, [1], []), [SystemColumnId(0); HT{ physical: 1690835538627274 }]) => null
  T ab874dea16d64a14892c2f3fea29a981 P ade1a2d814884e498b1f69a1d1be1041 [R]: 2. PutCF: SubDocKey(DocKey(0xeda9, [1], []), [ColumnId(1); HT{ physical: 1690835538627274 w: 1 }]) => "HitchHiker"
  T ab874dea16d64a14892c2f3fea29a981 P ade1a2d814884e498b1f69a1d1be1041 [R]: 3. PutCF: SubDocKey(DocKey(0xcfaa, [2], []), [SystemColumnId(0); HT{ physical: 1690835538627274 w: 2 }]) => null
  T ab874dea16d64a14892c2f3fea29a981 P ade1a2d814884e498b1f69a1d1be1041 [R]: 4. PutCF: SubDocKey(DocKey(0xcfaa, [2], []), [ColumnId(1); HT{ physical: 1690835538627274 w: 3 }]) => "Restaurant"
I0731 13:32:18.627890 46346240 tablet.cc:1495] T ab874dea16d64a14892c2f3fea29a981 P ade1a2d814884e498b1f69a1d1be1041: Wrote 15 key/value pairs to kIntents RocksDB:
  T ab874dea16d64a14892c2f3fea29a981 P ade1a2d814884e498b1f69a1d1be1041 [I]: 1. SingleDeleteCF: TXN META 45f1546f-1634-45a1-aa7a-80a80a0b84a4
  T ab874dea16d64a14892c2f3fea29a981 P ade1a2d814884e498b1f69a1d1be1041 [I]: 2. SingleDeleteCF: TXN REV 45f1546f-1634-45a1-aa7a-80a80a0b84a4 HT{ physical: 1690835538626171 }
  T ab874dea16d64a14892c2f3fea29a981 P ade1a2d814884e498b1f69a1d1be1041 [I]: 3. SingleDeleteCF: SubDocKey(DocKey(0xeda9, [1], []), [SystemColumnId(0)]) [kStrongRead, kStrongWrite] HT{ physical: 1690835538626171 }
  T ab874dea16d64a14892c2f3fea29a981 P ade1a2d814884e498b1f69a1d1be1041 [I]: 4. SingleDeleteCF: TXN REV 45f1546f-1634-45a1-aa7a-80a80a0b84a4 HT{ physical: 1690835538626171 w: 1 }
  T ab874dea16d64a14892c2f3fea29a981 P ade1a2d814884e498b1f69a1d1be1041 [I]: 5. SingleDeleteCF: SubDocKey(DocKey(0xeda9, [1], []), [ColumnId(1)]) [kStrongRead, kStrongWrite] HT{ physical: 1690835538626171 w: 1 }
  T ab874dea16d64a14892c2f3fea29a981 P ade1a2d814884e498b1f69a1d1be1041 [I]: 6. SingleDeleteCF: TXN REV 45f1546f-1634-45a1-aa7a-80a80a0b84a4 HT{ physical: 1690835538626171 w: 2 }
  T ab874dea16d64a14892c2f3fea29a981 P ade1a2d814884e498b1f69a1d1be1041 [I]: 7. SingleDeleteCF: SubDocKey(DocKey(0xcfaa, [2], []), [SystemColumnId(0)]) [kStrongRead, kStrongWrite] HT{ physical: 1690835538626171 w: 2 }
  T ab874dea16d64a14892c2f3fea29a981 P ade1a2d814884e498b1f69a1d1be1041 [I]: 8. SingleDeleteCF: TXN REV 45f1546f-1634-45a1-aa7a-80a80a0b84a4 HT{ physical: 1690835538626171 w: 3 }
  T ab874dea16d64a14892c2f3fea29a981 P ade1a2d814884e498b1f69a1d1be1041 [I]: 9. SingleDeleteCF: SubDocKey(DocKey(0xcfaa, [2], []), [ColumnId(1)]) [kStrongRead, kStrongWrite] HT{ physical: 1690835538626171 w: 3 }
  T ab874dea16d64a14892c2f3fea29a981 P ade1a2d814884e498b1f69a1d1be1041 [I]: 10. SingleDeleteCF: TXN REV 45f1546f-1634-45a1-aa7a-80a80a0b84a4 HT{ physical: 1690835538626171 w: 4 }
  T ab874dea16d64a14892c2f3fea29a981 P ade1a2d814884e498b1f69a1d1be1041 [I]: 11. SingleDeleteCF: SubDocKey(DocKey(0xcfaa, [2], []), []) [kWeakRead, kWeakWrite] HT{ physical: 1690835538626171 w: 4 }
  T ab874dea16d64a14892c2f3fea29a981 P ade1a2d814884e498b1f69a1d1be1041 [I]: 12. SingleDeleteCF: TXN REV 45f1546f-1634-45a1-aa7a-80a80a0b84a4 HT{ physical: 1690835538626171 w: 5 }
  T ab874dea16d64a14892c2f3fea29a981 P ade1a2d814884e498b1f69a1d1be1041 [I]: 13. SingleDeleteCF: SubDocKey(DocKey(0xeda9, [1], []), []) [kWeakRead, kWeakWrite] HT{ physical: 1690835538626171 w: 5 }
  T ab874dea16d64a14892c2f3fea29a981 P ade1a2d814884e498b1f69a1d1be1041 [I]: 14. SingleDeleteCF: TXN REV 45f1546f-1634-45a1-aa7a-80a80a0b84a4 HT{ physical: 1690835538626171 w: 6 }
  T ab874dea16d64a14892c2f3fea29a981 P ade1a2d814884e498b1f69a1d1be1041 [I]: 15. SingleDeleteCF: SubDocKey(DocKey([], []), []) [kWeakRead, kWeakWrite] HT{ physical: 1690835538626171 w: 6 }
```

In [ ]:
%%sql 
update demo set value='nada';

---

In [ ]:
%%sql 
drop table if exists tmp_pk_hash;
create temporary table if not exists tmp_pk_hash (k int, v1 text, v2 text, hash_code int, col_0x0000_0x5555 text ,col_0x5555_0xAAAA text,col_0xAAAA_0xFFFF text);

insert into tmp_pk_hash  (k,v1, v2, hash_code )
--, col_0x0000_0x5555,col_0x5555_0xAAAA,col_0xAAAA_0xFFFF)
select 
   g.id 
   , fn_random_chars()
   , fn_random_chars()
   , yb_hash_code(g.id)
   -- , fn_find_hash_code_in_partition_hex_range(yb_hash_code( g.id),'hash_split: [0x0000, 0x5555)'::text) as col_0x0000_0x5555
   -- , fn_find_hash_code_in_partition_hex_range(yb_hash_code( g.id),'hash_split: [0x5555, 0xAAAA)'::text) as col_0x5555_0xAAAA
   -- , fn_find_hash_code_in_partition_hex_range(yb_hash_code( g.id),'hash_split: [0xAAAA, 0xFFFF)'::text) as col_0xAAAA_0xFFFF
from generate_series(1, 10) AS g (id) ;

In [ ]:
%%sql 
drop table if exists tbl_pk_hash;
create table if not exists tbl_pk_hash (k int, v1 text, v2 text, hash_code int, col_0x0000_0x5555 text ,col_0x5555_0xAAAA text,col_0xAAAA_0xFFFF text);

insert into tbl_pk_hash  (k,v1,v2, hash_code , col_0x0000_0x5555,col_0x5555_0xAAAA,col_0xAAAA_0xFFFF)
select 
   k
   , v1
   , v2
   , yb_hash_code(v1)
   , fn_find_hash_code_in_partition_hex_range(yb_hash_code( v1),'hash_split: [0x0000, 0x5555)'::text) as col_0x0000_0x5555
   , fn_find_hash_code_in_partition_hex_range(yb_hash_code( v1),'hash_split: [0x5555, 0xAAAA)'::text) as col_0x5555_0xAAAA
   , fn_find_hash_code_in_partition_hex_range(yb_hash_code( v1),'hash_split: [0xAAAA, 0xFFFF)'::text) as col_0xAAAA_0xFFFF
from tmp_pk_hash 

--where col_0x5555_0xAAAA <> '';


In [ ]:
%%sql 
select 
  k
  , v1
  , v2
  , hash_code
  , col_0x0000_0x5555
  , col_0x5555_0xAAAA
  , col_0xAAAA_0xFFFF
from tbl_pk_hash
order by hash_code;

In [ ]:
%%sql 
create index
 idx_cols
 on tbl_pk_hash
(v1 hash,
 v2 asc);

http://127.0.0.1:7000


In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_TSERVER_WEBSERVER_PORT"  # yb-ctl remove node
YB_PATH=${1}
TSERVER_WEBSERVER_PORT=${2}

cd $YB_PATH

## add node
./bin/yb-ctl add_node \
--tserver_flags "enable_automatic_tablet_splitting=true,yb_num_shards_per_tserver=1,ysql_num_shards_per_tserver=1,ysql_beta_features=true,webserver_port="${TSERVER_WEBSERVER_PORT}  \
--num_shards_per_tserver=1  \
--placement_info "cloud1.region1.zone1" 



In [ ]:
%%sql 

insert into tbl_pk_hash  (k,v)
select g.id, fn_random_chars()
from generate_series(11, 2000000) AS g (id);

In [ ]:
%%sql 

insert into tbl_pk_hash  (k,v)
select g.id, fn_random_chars()
from generate_series(10000001,20000000) AS g (id);

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_TSERVER_WEBSERVER_PORT"  # yb-ctl remove node
YB_PATH=${1}
TSERVER_WEBSERVER_PORT=${2}

cd $YB_PATH
echo $YB_PATH
./bin/yb-admin --list

# .bin/yb-admin \
#     -master_addresses 127.0.0.1:7100 \
#     change_config f3cc5d37893f4e51a08b900cfda87219 \
#     [ ADD_SERVER | REMOVE_SERVER ] \
#     <peer_uuid> \
#     [ PRE_VOTER | PRE_OBSERVER ]


In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_TSERVER_WEBSERVER_PORT"  # yb-ctl remove node
YB_PATH=${1}
TSERVER_WEBSERVER_PORT=${2}

cd $YB_PATH
# ./bin/yb-ctl start_node 3 \
#       --master_flags "yb_num_shards_per_tserver=1,ysql_num_shards_per_tserver=1" \
#       --tserver_flags "yb_num_shards_per_tserver=1,ysql_num_shards_per_tserver=1,ysql_beta_features=true,webserver_port=8200" \
#       --num_shards_per_tserver 1  \
#       --placement_info "cloud3.region3.zone3" 

#./bin/yb-ctl start_node 3  --tserver_flags "yb_num_shards_per_tserver=1,ysql_num_shards_per_tserver=1,ysql_beta_features=true,webserver_port=8200" \
 --num_shards_per_tserver 1  \
   --placement_info "cloud1.region1.zone1,cloud2.region2.zone2,cloud3.region3.zone3" 
./bin/yb-ctl status

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_TSERVER_WEBSERVER_PORT"  # yb-ctl remove node
YB_PATH=${1}
TSERVER_WEBSERVER_PORT=${2}

cd $YB_PATH
./bin/yb-ctl start_node 3 \
      --master \
      --master_flags "yb_num_shards_per_tserver=1,ysql_num_shards_per_tserver=1" \
      --tserver_flags "yb_num_shards_per_tserver=1,ysql_num_shards_per_tserver=1,ysql_beta_features=true,webserver_port=8200" \
      --num_shards_per_tserver 1  \
      --placement_info "cloud3.region3.zone3" 


./bin/yb-ctl status

Index


In [ ]:
%%sql 
select 'a', yb_hash_code('a'::text)
union all
select 'b', yb_hash_code('b'::text)
union all
select 'c', yb_hash_code('c'::text)
union all
select 'd', yb_hash_code('d'::text)
union all
select 'e', yb_hash_code('e'::text)
union all
select 'f', yb_hash_code('f'::text)
union all
select 'g', yb_hash_code('g'::text)
union all
select 'h', yb_hash_code('h'::text)
union all
select 'i', yb_hash_code('i'::text)
union all
select 'j', yb_hash_code('j'::text)
union all
select 'k', yb_hash_code('k'::text)
order by 1;


In [ ]:
%%sql 
select v.c1 , yb_hash_code(v.c1::text)
from (
select 'a' as c1
union all
select 'b'
union all
select 'c'
union all
select 'd'
union all
select 'e'
union all
select 'f'
union all
select 'g'
union all
select 'h'
union all
select 'i'
union all
select 'j'
union all
select 'k'
) v
order by 1;

In [ ]:
%%sql 
select v.c1 
  , yb_hash_code(v.c1::text)
  , fn_find_hash_code_in_partition_hex_range(yb_hash_code(v.c1::text),'hash_split: [0x0000, 0x5555)'::text) as col_0x0000_0x5555
  , fn_find_hash_code_in_partition_hex_range(yb_hash_code(v.c1::text),'hash_split: [0x5555, 0xAAAA)'::text) as col_0x5555_0xAAAA
  , fn_find_hash_code_in_partition_hex_range(yb_hash_code(v.c1::text),'hash_split: [0xAAAA, 0xFFFF)'::text) as col_0xAAAA_0xFFFF
from (
select 'h' as c1
union all
select 'i'
union all
select 'j'
union all
select 'k'
union all 
select 'l'
union all
select 'm'
union all
select 'n'
union all
select 'o'
union all
select 'p'
union all
select 'q'
) v
order by 1;